## Import Library

In [4]:
from datetime import datetime

import logging
import re
import time
import sys

# Data preprocessing
import os
from collections import Counter
from nltk.tokenize import word_tokenize

# for transformer 
from soynlp.normalizer import repeat_normalize
from soynlp.normalizer import *
from soynlp.noun import NewsNounExtractor
from transformers import BertModel, BertTokenizer, AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import gluonnlp as nlp
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

# torch library
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from sklearn.metrics import f1_score, accuracy_score

import warnings
warnings.filterwarnings(action='ignore')

In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0")

## Data load & Preprocessing

In [6]:
# Data Loading to clustering

root_path = os.getcwd()
print(root_path)
dir_path = root_path +'/2-1. analysis dataset/esg_data_for_sentiment.csv'

df = pd.read_csv(dir_path)
#df = df.drop(['Unnamed: 0'], axis=1)
df.head()

/nas1/yongk/kpmg/BERT_for_finetuning


,date,text,sentiment
0,2020-01-01,가구보러 왔다가 책도 읽고 가지요신세계 리빙앤라이프스타일 브랜드 까사미아는 지난해 ...,1
1,2020-01-01,가구보러 왔다가 책도 읽고 가지요신세계 리빙앤라이프스타일 브랜드 까사미아는 지난해 ...,1
2,2020-01-02,재계 신년사 미래 고객 디지털 혁신 화두 그룹과 롯데 신세계 그룹은 고객 중심에 방...,0
3,2020-01-02,신년사 미래 고객 디지털 혁신 화두 그룹과 롯데 신세계 그룹은 고객 중심에 방점을 ...,0
4,2020-01-02,고체 식초 발포정 리아퐁 독보적인 기술력과 제품력으로 엔타스 면세점 입점고체 식초 ...,0


In [7]:
def preprocess_data(data, data_colname):
    """
      tips: csv 데이터를 받아 지정된 column의 내용을 preprocess 합니다.
      Args:
          data_path : csv데이터의 path
          data_colname : 지정할 column명
      Returns:
          lucy_data : DataFrame
    """
    lucy_data = data

    lucy_data[data_colname] = lucy_data[data_colname].str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("\[.*\]|\s-\s.*"," ",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("\<.*\>|\s-\s.*"," ",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("무단전재 및 재배포 금지"," ",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("무단 전재 및 재배포 금지"," ",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("©"," ",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("ⓒ"," ",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("저작권자"," ",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
    lucy_data[data_colname] = lucy_data[data_colname].str.replace('\"', "",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("\n"," ")
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("\r"," ")
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("\t"," ")
    lucy_data[data_colname] = lucy_data[data_colname].str.replace( "\’" , "", regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("[ ]{2,}"," ",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("?","",regex=True)
    
    return lucy_data

In [8]:
# Data preprocessing
# df의 title과 contents column을 합쳐서 text column을 만듭니다.
#df['text'] = df['title'] + df['contents']
clean_data = preprocess_data(df, 'text')
clean_data['text'] = clean_data['text'].str.replace(">"," ")
esg_data = clean_data

#esg_data = esg_data.drop(['Unnamed: 0'], axis=1)

# esg_data의 date, text, label column만 가져오기
esg_data = esg_data[['date', 'text', 'sentiment']]

In [9]:
esg_data
# sentiment label을 string으로
esg_data['sentiment'] = esg_data['sentiment'].astype(int)

In [10]:
# esg_data 개수 세기
esg_data['sentiment'].value_counts()


0    3504
1    2769
Name: sentiment, dtype: int64

In [11]:
def preprocess_data_2(data, data_colname):
    """
      tips: csv 데이터를 받아 지정된 column의 내용을 preprocess 합니다.
      Args:
          data_path : csv데이터의 path
          data_colname : 지정할 column명
      Returns:
          lucy_data : DataFrame
    """
    lucy_data = data

    lucy_data[data_colname] = lucy_data[data_colname].str.replace("?","",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace(".","0",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("20","2",regex=True)
    lucy_data[data_colname] = lucy_data[data_colname].str.replace("44","4",regex=True)
    
    return lucy_data

In [12]:
esg_data = preprocess_data_2(esg_data, 'sentiment')

AttributeError: Can only use .str accessor with string values!

In [ ]:
real_data_list = []
for date, q, label in zip(esg_data['date'], esg_data['text'], esg_data['sentiment']):
    data = []
    data.append(date)
    data.append(str(q))
    label = int(label)
    data.append(str(label))
    
    real_data_list.append(data)

### Config

In [37]:
# argment

max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate = 0.00003

### Set Dataloader

In [38]:
train_df, test_df = train_test_split(esg_data, test_size = 0.2, random_state=0, shuffle=True)

In [39]:
# train['text']를 string으로 변환
train_df['text'] = train_df['text'].astype(str)

In [40]:
# KoBERT Dataloader
class KoBERTDataset(Dataset):
    def __init__(self, dataset_text, dataset_date, dataset_label, bert_tokenizer, max_len,
                 pad, pair):
        
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.date = [[i] for i in dataset_date]
        self.text = [[i] for i in dataset_text]
        self.sentences = [transform([i]) for i in dataset_text]
        self.labels = [np.int32(i) for i in dataset_label]
        
    def __getitem__(self, i):
        return (self.sentences[i] + (self.text[i],) + (self.date[i],) + (self.labels[i],))
    
    def __len__(self):
        return (len(self.sentences))

In [41]:
# import pretrained kobert
from transformers import AutoTokenizer, AutoModelForMaskedLM
bertmodel, vocab = get_pytorch_kobert_model()
KoBERT_tokenizer = get_tokenizer()
BERT_tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert")
tok = nlp.data.BERTSPTokenizer(KoBERT_tokenizer, vocab, lower=False)

using cached model. /nas1/yongk/kpmg/BERT_for_finetuning/.cache/kobert_v1.zip
using cached model. /nas1/yongk/kpmg/BERT_for_finetuning/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /nas1/yongk/kpmg/BERT_for_finetuning/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [42]:
# KoBERT Dataloader

data_train = KoBERTDataset(train_df['text'], train_df['date'], train_df['sentiment'], tok, max_len, True, False)
data_test = KoBERTDataset(test_df['text'], test_df['date'], test_df['sentiment'], tok, max_len, True, False)
KoBERT_train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=5)
KoBERT_test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=True, num_workers=5)

## Modeling

In [43]:
class BERTClassifier_senti(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes = 2, dr_rate = 0.2, params = None):
        super(BERTClassifier_senti, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        
        return attention_mask.float()
    
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        #pooler = pooler.logits
        
        if self.dr_rate:
            out = self.dropout(pooler)
            
        return self.classifier(out)

### Utils

In [44]:
# Accuracy

def calc_accuracy(X, Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    
    return train_acc

In [45]:
# For training

class AverageMeter(object):
    def __init__(self, name):
        self.name = name
        self.reset()

    def reset(self):
        self.sum = 0
        self.count = 0
        self.avg = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = f'{self.name:10s} {self.avg:.8f}'
        return fmtstr

class ProgressMeter(object):
    def __init__(self, meters, loader_length, prefix=""):
        self.meters = [AverageMeter(i) for i in meters]
        self.loader_length = loader_length
        self.prefix = prefix
    
    def reset(self):
        for m in self.meters:
            m.reset()
    
    def update(self, values, n=1):
        for m, v in zip(self.meters, values):
            m.update(v, n)
            self.__setattr__(m.name, m.avg)

    def display(self, batch_idx, postfix=""):
        batch_info = f'[{batch_idx+1:03d}/{self.loader_length:03d}]'
        msg = [self.prefix + ' ' + batch_info]
        msg += [str(meter) for meter in self.meters]
        msg = ' | '.join(msg)

        sys.stdout.write('\r')
        sys.stdout.write(msg + postfix)
        sys.stdout.flush()

## KoBERT

In [46]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert")
kr_fin = AutoModelForMaskedLM.from_pretrained("snunlp/KR-FinBert")

In [47]:
model = BERTClassifier_senti(bertmodel, dr_rate = 0.2).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params':[p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params':[p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

t_total = len(KoBERT_train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

#loss_fn = nn.BCEWithLogitsLoss().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [48]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.best_epoch, self.best_accuracy = 0, 0
    
    def train(self, train_loader, epoch):
        progress = ProgressMeter(["train_loss", "train_acc"], len(train_loader), prefix=f'EPOCH {epoch:03d}')
        self.model.train()
        start_time = time.time()
        
        for batch_id, (token_ids, valid_length, segment_ids, text, date, label) in enumerate(tqdm_notebook(train_loader)):
            self.optimizer.zero_grad()
            
            label = label.unsqueeze(1)
            label = label.to(torch.int64)
            label = label.squeeze(dim=-1)
            label = label.long()
            
            token_ids, valid_length, segment_ids, label = token_ids.to(self.device), valid_length.to(self.device), segment_ids.to(self.device), label.to(self.device)
            logits = self.model(token_ids, valid_length, segment_ids)
            logits = logits.to(torch.float32) # torch.size([64, 7])
            
            loss = self.criterion(logits, label)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            self.optimizer.step()
            self.scheduler.step()
            #label = label.cpu().detach().numpy()
            #logits = logits.cpu().detach().numpy()
            
            acc = calc_accuracy(logits, label)
            #macro_f1 = f1_loss(label, logits)
            
            loss = loss.item()
            progress.update([loss, acc], n=token_ids.size(0))
            if batch_id % 20 == 0:
                progress.display(batch_id+1)
                
        self.scheduler: self.scheduler.step()
        finish_time = time.time()
        epoch_time = finish_time - start_time
        progress.display(batch_id, f' | {epoch_time:.0f}s' + '\n')
        
    def validate(self, val_loader, epoch):
        progress = ProgressMeter(["val_loss", "val_acc"], len(val_loader), prefix=f'VALID {epoch:03d}')
        self.model.eval()
        
        with torch.no_grad():
            for batch_id, (token_ids, valid_length, segment_ids, text, date, label) in enumerate(tqdm_notebook(val_loader)):
                
                label = label.unsqueeze(1)
                label = label.to(torch.int64)
                label = label.squeeze(dim=-1)
                label = label.long()
                
                token_ids, valid_length, segment_ids, label = token_ids.to(self.device), valid_length.to(self.device), segment_ids.to(self.device), label.to(self.device)
                logits = self.model(token_ids, valid_length, segment_ids)
            
                logits = logits.to(torch.float32)
                
                loss = self.criterion(logits, label)
                
                acc = calc_accuracy(logits, label)
                #macro_f1 = f1_score(label, logits)
                progress.update([loss, acc], n=token_ids.size(0))
            
            progress.display(batch_id, '\n')
            
    def test(self, test_loader):
        progress = ProgressMeter(["test_loss", "test_acc"], len(test_loader), prefix=f'TEST')
        #ckpt = torch.load(self.output_path + '/ckpt.pt')
        #self.model.load_state_dict(ckpt['model_state_dict'])
        self.model.eval()

        with torch.no_grad():
            for batch_id, (token_ids, valid_length, segment_ids, text, date, label) in enumerate(tqdm_notebook(test_loader)):
                label = label.unsqueeze(1)
                label = label.to(torch.int64)
                label = label.squeeze(dim=-1)
                label = label.long()
                
                token_ids, valid_length, segment_ids, label = token_ids.to(self.device), valid_length.to(self.device), segment_ids.to(self.device), label.to(self.device)
                
                logits = self.model(token_ids, valid_length, segment_ids)
                logits = logits.to(torch.float32)
                
                loss = self.criterion(logits, label)
                
                acc = calc_accuracy(logits, label)
                #macro_f1 = f1_score(label, logits, zero_division='warn', average='macro')
                progress.update([loss, acc], n=token_ids.size(0))
            progress.display(batch_id, '\n')
            torch.save(self.model, root_path + '/2-1. analysis dataset/esg_sent_finetuning.pt')

In [49]:
# trainer config define
trainer = Trainer(model, loss_fn, optimizer, scheduler, device)

In [50]:
# train & test
for epoch in range(num_epochs):
        trainer.train(KoBERT_train_loader, epoch)

  0%|          | 0/79 [00:00<?, ?it/s]

EPOCH 000 [079/079] | train_loss 0.42571477 | train_acc  0.79453966 | 34s


  0%|          | 0/79 [00:00<?, ?it/s]

EPOCH 001 [079/079] | train_loss 0.16940879 | train_acc  0.94400159 | 32s


In [51]:
# test traiend kobert
test_esg = trainer.test(KoBERT_test_loader)
test_esg

  0%|          | 0/20 [00:00<?, ?it/s]

TEST [020/020] | test_loss  0.14257644 | test_acc   0.95697211


In [ ]:
# load model
esg_sent_model = torch.load(root_path + '/2-1. analysis dataset/esg_sent_finetuning.pt')

In [183]:
# pre_label 이라는 column 2개를 가진 dataframe 생성 
pre_label = pd.DataFrame()
pre_text = pd.DataFrame()
pre_date = pd.DataFrame()

for batch_id, (token_ids, valid_length, segment_ids, text, date) in enumerate(tqdm_notebook(KoBERT_test_loader)):

    text_list = []
    pre_index_list = []
    date_list = []
    
    text = list(text)
    date = list(date)
    token_ids, valid_length, segment_ids = token_ids.to(device), valid_length.to(device), segment_ids.to(device)
                
    logits = esg_sent_model(token_ids, valid_length, segment_ids)
    logits = logits.to(torch.float32)
    pre_index = torch.argmax(logits, dim=1)
    for i in range(pre_index.shape[0]):
        text_list.append(text[i])
        date_list.append(date[i])
        pre_index_list.append(int(pre_index[i].cpu().detach().numpy()))
    # pre_index_list를 pre_label dataframe에 추가       
    pre_label = pre_label.append(pd.DataFrame(pre_index_list, columns=['pre_label']))
    pre_text = pre_text.append(pd.DataFrame(text_list, columns=['text']))
    pre_date = pre_date.append(pd.DataFrame(date_list, columns=['date']))


  0%|          | 0/43 [00:00<?, ?it/s]

In [186]:
# pre_label과 pre_text를 concat 
pre_df = pd.concat([pre_text, pre_label], axis=1)